# Tarefa 3: Executar o processamento de dados com o SageMaker Processing e o contêiner scikit-learn integrado

Neste notebook, você vai configurar o ambiente necessário para executar um script scikit-learn usando uma imagem do Docker fornecida e mantida pelo SageMaker Processing. 

Depois, você vai usar a classe **SKLearnProcessor** do SDK Python do SageMaker para definir e executar um trabalho de processamento do scikit-learn.

Por fim, você vai validar os resultados do processamento de dados salvos no Amazon Simple Storage Service (Amazon S3).

**Atenção:** o script de processamento faz um processamento básico de dados, como remoção de duplicações, transformação da coluna de destino em uma coluna contendo dois rótulos, codificação one-hot e uma divisão 80-20 para gerar conjuntos de dados de treinamento e teste. 

## Tarefa 3.1: Configuração do ambiente

Nesta tarefa, você vai instalar os pacotes e as dependências necessários. 

Você vai definir um bucket do Amazon S3 para armazenar as saídas do trabalho de processamento e fazer com que a função de execução execute o trabalho do SageMaker Processing.

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker.s3 import S3Downloader

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

## Tarefa 3.2: Executar o trabalho de processamento do SageMaker

Nesta tarefa, você vai importar e examinar o conjunto de dados pré-processado.

In [ ]:
#import-data
prefix = 'data/input'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/adult_data.csv", local_path= 'data/')

shape=pd.read_csv("data/adult_data.csv", header=None)
shape.sample(5)

Neste laboratório, são executadas as transformações de dados, como a remoção de duplicações, a transformação da coluna de destino em um coluna contendo dois rótulos e a codificação one-hot dos recursos categóricos.

Depois, você vai criar a classe SKLearnProcessor para definir e executar um script de processamento scikit-learn como um trabalho de processamento. Consulte [SKLearnProcessor scikit-learn do SageMaker](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#sagemaker.sklearn.processing.SKLearnProcessor) para saber mais sobre essa classe.

Para criar a classe SKLearnProcessor, configure os seguintes parâmetros:
- **base_job_name**: prefixo do nome do trabalho de processamento
- **framework_version**: versão do scikit-learn
- **role**: função de execução do SageMaker
- **instance_count**: número de instâncias para executar o trabalho de processamento
- **instance_type**: tipo de instância do Amazon Elastic Compute Cloud (Amazon EC2) usado para o trabalho de processamento

In [ ]:
#scikit-learn-processor
import os
from sagemaker.sklearn.processing import SKLearnProcessor

# create a SKLearnProcessor
sklearn_processor = SKLearnProcessor(
    base_job_name="sklearn-preprocessor",
    framework_version="0.20.0", 
    role=role, 
    instance_count=1,
    instance_type="ml.m5.xlarge"
)


Depois, use o método **SKLearnProcessor.run()** para executar o script **sklearn_preprocessing.py** como um trabalho de processamento. 

Para executar o trabalho de processamento, configure os seguintes parâmetros:
- **code**: caminho do script de pré-processamento 
- **inputs e outputs**: caminho da entrada e da saída do script de pré-processamento (local de entrada e de saída do Amazon S3)
- **arguments**: argumentos da linha de comando para o script de pré-processamento (como a taxa de divisão do teste e do treinamento)

O trabalho de processamento exige cerca de quatro a cinco minutos para ser concluído. Enquanto o trabalho estiver em execução, examine a origem do script de pré-processamento (que foi pré-configurado como parte deste laboratório) abrindo o arquivo **sklearn_preprocessing.py** com o navegador de arquivos.

In [ ]:
#processing-job
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
sklearn_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "adult_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                         source="/opt/ml/processing/test",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

print("SKLearn Processing Job Completed.")

## Tarefa 3.3: Validar os resultados do processamento de dados

Nesta tarefa, você vai validar a saída do trabalho de processamento que executou examinando as cinco primeiras linhas dos conjuntos de dados de saída de treinamento e teste.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

### Conclusão

Parabéns! Você usou o SageMaker Processing para criar um trabalho de processamento scikit-learn usando o SDK Python do SageMaker e executou o trabalho de processamento com sucesso.

A próxima tarefa do laboratório se concentra no processamento de dados usando o SageMaker Processing com o seu contêiner de processamento.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue na **Tarefa 4: Executar o processamento de dados com o seu contêiner**.